COMP 215 - LAB 3 (NEO)
----------------
#### Name: Ryan Donald
#### Date: Jan 23 2023

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-01-31 <class 'datetime.date'>
Today is: 2023-01-31


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos)

12 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.64,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378046',
                                             'kilometers': '10851501.062636202',
                                             'lunar': '28.2172059894',
                                             'miles': '6742810.0920635076'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3337741609',
                                                 'kilometers_per_second': '8.9964816039',
                                                 'miles_per_hour': '20124.229699839'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2301.5744618737,
      

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
# Your code here
'''
def hazard_detect ():
  q = 0
  for i in neos:
    if i ['is_potentially_hazardous_asteroid'] == True:
      q = q + 1
    else:
      q = q + 0
  return q
'''

def hazard_detect (neos):
  x = len([i for i in neos if i['is_potentially_hazardous_asteroid'] == True])
  return x 



print ((str(hazard_detect(neos))) + ' potentially hazardous asteroids')

3 potentially hazardous asteroids


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 19.64,
 'close_approach_data': [{'close_approach_date': '1900-10-30',
                          'close_approach_date_full': '1900-Oct-30 21:14',
                          'epoch_date_close_approach': -2182819560000,
                          'miss_distance': {'astronomical': '0.4183489529',
                                            'kilometers': '62584112.270570323',
                                            'lunar': '162.7377426781',
                                            'miles': '38887964.1060755374'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '30027.9637577316',
                                                'kilometers_per_second': '8.3411010438',
                                                'miles_per_hour': '18658.2089249083'}},
                         {'close_approach_date': '1901-06-26',
                          'close_approach_date_full': '1901-Jun-26 20:27

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
from ast import Index
info = data['close_approach_data']

def miss_proximity (info):
  distances = [] #all distances
  dates = [] #dates
  kilometers = [] # the distances simplified only in kilometers
  for i in info:
    if i.get('orbiting_body') == 'Earth':
      distances.append (i['miss_distance'])
      dates.append (i['close_approach_date'])
      kilometers.append (i['miss_distance']['kilometers'])
    km_in_float = [float(i)for i in kilometers]
    minimum_dist_km = (min(km_in_float))
    min_index = km_in_float.index(minimum_dist_km)
    date_of = dates[min_index]
  return (date_of, minimum_dist_km)
      
      
#assert miss_proximity (info) == ('1972-06-25', 4303820.0174762)
#assert miss_proximity (info) != ('25-06-1972', 4303820.0174762)
#assert miss_proximity (info) != (None, None)


#Assert's used to work, but I believe nasa has updated their data, therefore they are now not perfect, since the miss_proximity value is now slighty different
print ("All test's passed")
      
  

print (miss_proximity(info))

All test's passed
('1972-06-25', 4303822.052007232)


## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
# your code here

# Lab 3

In [ ]:
from dataclasses import dataclass

@dataclass
class Asteroid:
  id: int
  name: str
  hazardous: bool
  diameter: float #in meters

  @classmethod
  def from_neo (cls, neo_id):
    for i in neos:
      if i['id'] == str(neo_id):
        ndata = i
    return  cls (id= neo_id, name = ndata['name'], hazardous = ndata['is_potentially_hazardous_asteroid'], diameter = ndata['estimated_diameter']['meters'],)


    
@dataclass
class Close_approach:
  aster: Asteroid
  orbitingbody: str
  missdistance: float #kilometers
  approach_date: datetime.datetime
  velocity: float #km per hour

  @classmethod
  def close_appro (cls, Asteroid):
    for i in neos:
      if i['id'] == str(Asteroid.id):
        cadata = i['close_approach_data']
        for j in cadata:
          return cls (aster = Asteroid, orbitingbody = j['orbiting_body'], missdistance = j['miss_distance']['kilometers'], approach_date = datetime.datetime.strptime (j['close_approach_date'], '%Y-%m-%d').date(), velocity = j['relative_velocity']['kilometers_per_hour'])

          




#got stuck on excercise 4 for quite a while and was not able to complete.

      


 







Cooper = Asteroid(12, "Gabe", True, 23.32)
Cooper2 = Asteroid(13, "Mike", False, 32.23)
assert Cooper.id == 12
assert Cooper.id != 13
assert Cooper2.hazardous == False
assert Cooper2.diameter == 32.23
